In [7]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [8]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "round summary" in dict.keys():
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_36.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


You arrived at an old self-storage facility by invitation from the manager, who has an storage room up for auction due to the death of its owner. It is not your first rodeo with these forms of auctions, like a box of chocolate, no one knows of the potential treasure that hides behind the cold roller shutter. As you and other interested buyers gathered around the auctioneer, a worker carrying a bolt cutter snapped off the rusty lock, unveiling a storage room cluttered with various old furniture and antiquities. Without any opportunity for the bidders to take a clear look, the auctioneer began the bid at $50. 

What is your bid?

possible actions:
1. $100.
2. $150.
3. $200.


I select option 2. TURN: 1 


You raise your hand confidently and shout, "$150!" Your voice cuts through the murmurs of the crowd, drawing attention to your determined stance. The auctioneer nods in your direction, acknowledging your bid. 

"$150, do I hear more?" the auctioneer calls out, scanning the crowd for any further interest. The tension is palpable as eyes dart around, with some bidders whispering to each other, calculating their next move. 

possible actions:
1. Increase your bid to maintain the lead.
2. Hold your bid at $150 and wait to see if anyone else bids higher.
3. Examine the storage room from your spot to see if you can spot anything of value or interest.

I select option 1. TURN: 2 


Feeling the competitive spirit surge within you, you decide to increase your bid to maintain the lead. With a steady voice, you call out, "$200!" ensuring that your presence and determination are felt by everyone around. The auctioneer's eyes light up as he repeats your bid for all to hear, "$200, we have $200! Do I hear more?"

Your bold move has clearly made some of the other bidders hesitant, as they exchange glances, trying to decide whether to challenge your bid. The air is thick with anticipation, and the murmurs have quieted down, leaving the sound of the auctioneer's voice echoing in the open space.

possible actions:
1. Keep your bid at $200 and observe the crowd's reaction.
2. Prepare to increase your bid if someone else tries to outbid you.
3. Use this moment to closely observe the storage room's contents from your position, looking for any signs of hidden treasures.

I select option 1. TURN: 3 


You hold your ground at $200, watching the crowd closely for their reaction. Your assertiveness seems to have made an impact, as the other bidders appear reluctant to challenge your bid immediately. The auctioneer, sensing the hesitation, tries to encourage more bidding, "We have $200! Can I get $250? Don't miss out on this opportunity, folks!"

As the seconds tick by, the crowd remains silent, with no one stepping forward to outbid you. It seems your gamble might pay off, but the auction isn't over yet. The auctioneer gives it one last try, scanning the crowd for any signs of interest.

possible actions:
1. Stand firm with your bid at $200, waiting for the auctioneer to call it.
2. Consider raising your bid preemptively to discourage any last-minute bids.
3. Use the remaining time to try and get a better look at some of the items in the storage room, looking for anything that might justify your bid.

I select option 1. TURN: 4 


You decide to stand firm with your bid at $200, letting the tension in the air build as the auctioneer makes one final appeal for higher bids. "Last chance, folks! $200 going once, going twice..."

The crowd remains silent, no hands go up, and no voices challenge your bid. It's as if the collective breath of the onlookers is held in suspense, waiting for the final hammer to fall.

"Sold to the gentleman for $200!" the auctioneer finally declares, pointing in your direction. A mixture of relief and excitement washes over you as you realize you've just secured the contents of the storage room. Now, the real mystery begins - what treasures or secrets might lie within?

round summary: Kyle Jackson confidently entered the auction for a mysterious storage room, starting with a bid of $150 and strategically raising it to $200 amidst the competitive atmosphere. His boldness paid off, as no other bidders challenged his final bid, allowing him to win the auction. No

...

I select option 3. TURN: 1 


As you choose to confront Alex with a barrage of verbal assaults, your words, sharp and filled with venom, cut through the tense air. However, Alex, unfazed by your attempt to intimidate or dissuade him with words, stands towering over you, his fists clenched, eyes burning with a mix of anger and impatience.

possible actions:
1. Attempt to calm the situation by offering a partial payment.
2. Get up and prepare to defend yourself.
3. Try to reason with Alex, mentioning the peculiar wooden block as a potential high-value item.

I select option 2. TURN: 2 


Gathering your strength, you push yourself off the ground, standing tall in front of Alex. Your stance is defensive, ready to parry or dodge any incoming attacks. You recall your training from the Marines, focusing on staying alert and prepared for whatever Alex might throw at you.

possible actions:
1. Throw a punch aiming for Alex's face.
2. Attempt a quick maneuver to get behind Alex.
3. Keep your position, trying to de-escalate the situation verbally while staying on guard.

I select option 1. TURN: 3 


With a burst of adrenaline, you launch a swift punch aimed directly at Alex's face, hoping to catch him off guard and maybe knock some sense into him. Your fist cuts through the air, targeting his jaw with precision honed from your military training.

skill: fighting
difficulty: hard

I am making a skill check using FIGHTING against a difficulty of HARD.
And I rolled a 33 for a result of FAILURE. TURN: 4 


Your punch, though fast and aimed with precision, misses Alex as he steps back just in time, avoiding your strike. The miss leaves you slightly off-balance, and Alex, seizing the opportunity, lands a heavy blow to your side. The impact forces the air out of your lungs, and pain radiates through your ribs. Despite the setback, you manage to stay on your feet, though the pain is a clear reminder of the dire situation.

Alex, seeing your resilience, pulls out a handgun from his jacket, his intentions clear. He doesn't aim it at you yet, but the message is unmistakable: he's not messing around, and he's willing to escalate this confrontation to get what he wants.

possible actions:
1. Try to calm the situation, acknowledging his seriousness.
2. Make a strategic retreat, putting some distance between you and the gun.
3. Attempt to disarm him, using your close combat t

I select option 1. TURN: 5 


Recognizing the gravity of the situation and the imminent threat posed by the handgun, you raise your hands in a gesture of surrender, your voice steady as you try to calm the escalating tension. "Alright, Alex, I get it," you say, attempting to reason with him and acknowledge the seriousness of his actions. "You'll get your money, I promise."

Alex, his expression hardening as he assesses your sincerity, slowly lowers the gun, though his grip remains tight. "You have until tomorrow," he warns, his voice low and menacing. "Don't make me come looking for you again."

With that, Alex turns and walks away, leaving you standing there, bruised and shaken, but alive. The encounter leaves you with a newfound urgency to settle your debt with Alex, and the peculiar wooden block in your possession suddenly seems like a key piece in this dangerous puzzle.

round summary: Kyle attempted to verbally assault Alex, which escalated the confrontation. Kyle then prepared to